# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establiments --drop --jsonArray establishments.json`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'classDB', 'config', 'epa', 'fruits_db', 'gardendb', 'local', 'met', 'petsitly_marketing', 'travel_db', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [6]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'The Pines Garden',
 'AddressLine2': 'Beach Road',
 'AddressLine3': 'St Margarets Bay',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pines Calyx',
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4587.362402580997,
 'FHRSID': 254250,
 'LocalAuthorityBusinessID': 'PI/000066174',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DZ',
 'RatingDate': '2021-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6413c91a276f36bac44dd4ab'),
 'geocode': {'latitude': '51.148133', 'longitude': '1.383298'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254250',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extract

In [7]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
rest_dict = dict()

In [9]:
# Insert the new restaurant into the collection
rest_dict = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [10]:
# Check that the new restaurant was inserted
rest_dict

{'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'PostCode': 'SE18 7DY',
 'Phone': '',
 'LocalAuthorityCode': '511',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'},
 'RightToReply': '',
 'Distance': 4623.972328074718,
 'NewRatingPending': True}

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [19]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields

query = {'BusinessType': 'Restaurant/Cafe/Canteen'}
fields = {'BusinessTypeID', 'BusinessType'}
results = list(establishments.find(query, fields))
pprint(results[1])

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('6413c91a276f36bac44dd4ad')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [20]:
# Update the new restaurant with the correct BusinessTypeID
db.establishments.update_one(
    {'BusinessName': 'Penang Flavours'},
    {'$set':
        {'BusinessTypeID': 1}
    }
)

In [25]:
# Confirm that the new restaurant was updated
query = {'BusinessName': 'Penang Flavours'}
results = establishments.find(query)
for result in results:
    print(result)

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [27]:
# Find how many documents have LocalAuthorityName as "Dover"

query = {'LocalAuthorityName': 'Dover'}

results = establishments.find(query)

establishments.count_documents(query)


994

In [28]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many(query)

In [29]:
# Check if any remaining documents include Dover
establishments.count_documents(query)

0

In [31]:
# Check that other documents remain with 'find_one'
pprint(db.establishments.find_one())

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6413c91a276f36bac44dd791'),
 'geocode': {'latitude': '51.086058', 'longitude': '1.196408'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          '

5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [34]:
# Change the data type from String to Decimal for longitude
establishments.update_many({}, [ {'$set':{ "geocode.longitude" : {'$toDecimal': "$geocode.longitude"}}} ])

In [35]:
# Change the data type from String to Decimal for latitude
establishments.update_many({}, [ {'$set':{ "geocode.latitude" : {'$toDecimal': "$geocode.latitude"}}} ])

In [36]:
# Check that the coordinates are now numbers
query = {}
fields = {'geocode.latitude': 1, 'geocode.longitude': 1}

results = list(establishments.find(query, fields))
pprint(results[0:10])

[{'_id': ObjectId('6413c91a276f36bac44dd791'),
  'geocode': {'latitude': Decimal128('51.086058'),
              'longitude': Decimal128('1.196408')}},
 {'_id': ObjectId('6413c91a276f36bac44dd792'),
  'geocode': {'latitude': Decimal128('51.085797'),
              'longitude': Decimal128('1.194762')}},
 {'_id': ObjectId('6413c91a276f36bac44dd796'),
  'geocode': {'latitude': Decimal128('51.083812'),
              'longitude': Decimal128('1.195625')}},
 {'_id': ObjectId('6413c91a276f36bac44dd797'),
  'geocode': {'latitude': Decimal128('51.08084'),
              'longitude': Decimal128('1.188537')}},
 {'_id': ObjectId('6413c91a276f36bac44dd798'),
  'geocode': {'latitude': Decimal128('51.08084'),
              'longitude': Decimal128('1.188537')}},
 {'_id': ObjectId('6413c91a276f36bac44dd799'),
  'geocode': {'latitude': Decimal128('51.0783519967076'),
              'longitude': Decimal128('1.18590330311705')}},
 {'_id': ObjectId('6413c91a276f36bac44dd79a'),
  'geocode': {'latitude': Decimal1